## Day 68 – MLOps Model Versioning

## Introduction

In this notebook, I continued my MLOps learning journey by exploring **Model Versioning** using **MLflow**.  
While Day 67 focused on experiment tracking, this session focuses on how to **store, organize, and manage multiple versions** of trained models efficiently.

Model versioning is a critical step in MLOps because:
- It allows tracking how a model evolves over time.
- Ensures reproducibility of results across versions.
- Simplifies deployment by maintaining a clear record of model changes.

Here, I trained multiple models and used **MLflow’s Model Registry** to save and register each version for comparison and management.

---

## What is Model Versioning?

**Model Versioning** means assigning and tracking unique versions to machine learning models as they evolve.

Just like software has version control (v1.0, v2.0, etc.), in MLOps we track:
- **Model Versions** – each trained model instance (e.g., logistic regression v1, random forest v2)
- **Parameters and Metrics** – what changed between versions
- **Experiment Artifacts** – saved models, requirements, and configuration files

### Why it matters:
- Keeps your experiments **organized and reproducible**
- Enables **rollbacks** to older models if newer ones perform worse
- Facilitates **team collaboration** by identifying which model is currently in production

---

## MLflow Model Versioning Workflow

MLflow’s **Model Registry** provides a central hub to manage models and their lifecycle stages.

### Typical Workflow:

1. **Train and log models** using MLflow (`mlflow.sklearn.log_model()`).
2. **Register models** in the MLflow Model Registry with `mlflow.register_model()`.
3. **Assign version numbers** automatically (v1, v2, v3, ...).
4. **Promote models** through lifecycle stages:
   - *Staging* → model is under testing  
   - *Production* → approved for deployment  
   - *Archived* → old or unused models
5. **Load models** by version or stage for inference using `mlflow.pyfunc.load_model()`.

Each new model logged to the same registry name automatically increments its version.

---

## Import Required Libraries

I begin by importing the necessary packages:
- **NumPy, Scikit-learn** → data creation and model training  
- **MLflow & MLflow.sklearn** → for tracking and versioning models  
- **make_classification** → generates sample data for quick testing  

MLflow helps track every run automatically, including metrics and saved models.


In [1]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import mlflow
import mlflow.sklearn
import mlflow.xgboost
import warnings
warnings.filterwarnings('ignore')

# Data Preparation and Model Training

I create a **synthetic dataset** using `make_classification()` and train a **Logistic Regression model**.  
This helps simulate real-world training scenarios quickly.

After training, I log parameters, metrics, and models to MLflow just like in Day 67 — but now, I will also **register the model** into the MLflow Model Registry.


In [2]:
# Step 1: Create an imbalanced binary classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100], dtype=int64))

In [3]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

#### Handle class imbalance

In [4]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)
np.unique(y_train_res, return_counts=True)

(array([0, 1]), array([619, 619], dtype=int64))

### Track Experiments

In [5]:
models = [
    (
        "Logistic Regression", 
        {"C": 1, "solver": 'lbfgs'},
        LogisticRegression(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "Random Forest", 
        {"n_estimators": 30, "max_depth": 3},
        RandomForestClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier With SMOTE",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train_res, y_train_res),
        (X_test, y_test)
    )
]

In [6]:
reports = []

for model_name, params, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]
    
    model.set_params(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

In [8]:
# Initialize MLflow
mlflow.set_experiment("deployment model")
mlflow.set_tracking_uri("http://localhost:5000")

for i, element in enumerate(models):
    model_name = element[0]
    params = element[1]
    model = element[2]
    report = reports[i]
    
    with mlflow.start_run(run_name=model_name):        
        mlflow.log_params(params)
        mlflow.log_metrics({
            'accuracy': report['accuracy'],
            'recall_class_1': report['1']['recall'],
            'recall_class_0': report['0']['recall'],
            'f1_score_macro': report['macro avg']['f1-score']
        })  
        
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")  

2025/11/11 23:37:14 INFO mlflow.tracking.fluent: Experiment with name 'deployment model' does not exist. Creating a new experiment.
2025/11/11 23:37:17 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/11 23:37:25 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/11/11 23:37:27 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run Logistic Regression at: http://localhost:5000/#/experiments/793610528629521495/runs/6fe54ecb39c8432b84861c2b59adf253
🧪 View experiment at: http://localhost:5000/#/experiments/793610528629521495


2025/11/11 23:37:32 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/11/11 23:37:35 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run Random Forest at: http://localhost:5000/#/experiments/793610528629521495/runs/8cd402a08cd245ea8cde81cbff76bf7c
🧪 View experiment at: http://localhost:5000/#/experiments/793610528629521495


2025/11/11 23:37:42 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/11/11 23:37:45 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run XGBClassifier at: http://localhost:5000/#/experiments/793610528629521495/runs/c76bb759d7ed468e906728401c270f72
🧪 View experiment at: http://localhost:5000/#/experiments/793610528629521495


2025/11/11 23:37:51 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier With SMOTE at: http://localhost:5000/#/experiments/793610528629521495/runs/29df1c4b0660462cb367cc447a07b3ba
🧪 View experiment at: http://localhost:5000/#/experiments/793610528629521495


### Experiments Dashboard

![Experiments Dashboard](screenshots/screenshot1.png)

**Explanation:**
This is the **main MLflow Experiments dashboard**. It displays all experiments created so far — such as `deployment model`, `1st Experiment`, and `Default`. Each experiment acts as a separate workspace to organize and track multiple model runs.

---

### Runs List under “deployment model” Experiment

![deployment model Experiment](screenshots/screenshot2.png)

**Explanation:**
Here we can see **four model runs** under the “deployment model” experiment — Logistic Regression, Random Forest, XGBoost, and XGBoost with SMOTE.

Each row represents a model training run with its **runtime, source, and tracked metrics**.
This helps in comparing multiple algorithms on the same dataset.

---

### Comparing 4 Runs from 1 Experiment

![Comparing 4 Runs](screenshots/screenshot3.png)

**Explanation:**
This visualization shows how MLflow allows you to **compare multiple runs** within the same experiment.

The **Parallel Coordinates Plot** compares model performance based on accuracy. It helps to quickly identify which model performed best across key metrics.

---

### Detailed Run Comparison

![Detailed Run Comparison](screenshots/screenshot4.png)

**Explanation:**
This comparison table provides a **side-by-side summary** of all selected model runs.

It includes:

* **Run ID, model name, duration**
* **Metrics** such as accuracy, F1 score, and recall for each class
  From this view, it’s clear which model achieves the best results.

---

### Models Dashboard

![Models Dashboard](screenshots/screenshot5.png)

**Explanation:**
This is the **Models section** in MLflow UI.
Here you can create, view, and manage **registered models**.

At this stage, no models are registered yet — we will create one in the next step.

---

### Creating the Model

![Creating the Model](screenshots/screenshot6.png)

**Explanation:**
This dialog box appears when you click **“Create Model”** in MLflow.

Here, the model name `XGB-Smote` is entered — which will be used to register the best-performing model from the experiments.

---

### Model Created Successfully

![Created Model](screenshots/screenshot7.png)

**Explanation:**
After creation, the model `XGB-Smote` now appears under **Registered Models**.
This confirms that the model registry entry has been created successfully.

---

### Registered Model Details

![Registered Model Details](screenshots/screenshot8.png)

**Explanation:**
This section shows detailed information about the **registered model** — including creation time, description, and available versions.

Currently, there are **no versions yet**, but once models are registered through the code, each version will appear here (v1, v2, etc.) with its corresponding metadata.

---



## Register Model in MLflow Model Registry

Once a model is logged, we can register it using `mlflow.register_model()`.  
Example:
```python
mlflow.register_model("runs:/<run_id>/model", "MyModelName")
````

This command:
- Links the model artifact from your current MLflow run.
- Creates a named entry (e.g., “MyModelName”) inside the Model Registry.
- Automatically assigns a version number — Version 1 for the first model.

Each time you register a new model under the same name, MLflow creates a new version (Version 2, 3, etc.).

In [9]:
model_name = 'XGB-Smote'
run_id=input('Please type RunID')
model_uri = f'runs:/{run_id}/model'

with mlflow.start_run(run_id=run_id):
    mlflow.register_model(model_uri=model_uri, name=model_name)

Registered model 'XGB-Smote' already exists. Creating a new version of this model...
2025/11/11 23:47:46 WARNING mlflow.tracking._model_registry.fluent: Run with id 29df1c4b0660462cb367cc447a07b3ba has no artifacts at artifact path 'model', registering model based on models:/m-2db6cfc9c32b417795e2165dbff0e4c1 instead
2025/11/11 23:47:46 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XGB-Smote, version 1
Created version '1' of model 'XGB-Smote'.


🏃 View run XGBClassifier With SMOTE at: http://localhost:5000/#/experiments/793610528629521495/runs/29df1c4b0660462cb367cc447a07b3ba
🧪 View experiment at: http://localhost:5000/#/experiments/793610528629521495


### After Registering the Model (Version 1 Created)

![Version 1](screenshots/screenshot9.png)

**Explanation:**
This screenshot shows that the model **`XGB-Smote`** has been successfully **registered** in the MLflow Model Registry.

A new **Version 1** entry appears, showing details such as:

* Registration time
* Model creator
* Stage and description options
  This confirms the first version of the model is stored and ready for promotion or deployment.

---

### After Adding Aliases

![Aliases](screenshots/screenshot10.png)

**Explanation:**
Here, an **alias named `@appserver`** has been added to Version 1 of the `XGB-Smote` model.
Aliases act as **labels** for specific versions (e.g., `@staging`, `@production`, `@test`) to make deployment references easier.

This helps teams refer to models dynamically without hardcoding version numbers.

---

### Load the Model

In [10]:
model_name = 'XGB-Smote'
model_version = 1
model_uri = f"models:/{model_name}/{model_version}"

loaded_model = mlflow.xgboost.load_model(model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

array([0, 0, 0, 0])

### Transition the Model to Production server

In [11]:
current_model_uri = f"models:/{model_name}@appserver"
production_model_name = "finalproduction"

client = mlflow.MlflowClient()
client.copy_model_version(src_model_uri=current_model_uri, dst_name=production_model_name)

Successfully registered model 'finalproduction'.
Copied version '1' of model 'XGB-Smote' to version '1' of model 'finalproduction'.


<ModelVersion: aliases=[], creation_timestamp=1762885436220, current_stage='None', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='', last_updated_timestamp=1762885436220, metrics=None, model_id=None, name='finalproduction', params=None, run_id='29df1c4b0660462cb367cc447a07b3ba', run_link='', source='models:/XGB-Smote/1', status='READY', status_message=None, tags={}, user_id='', version='1'>

### Final Production Server

![Final Production Server](screenshots/screenshot11.png)

**Explanation:**
This view shows the **final list of registered models**, including `XGB-Smote` and `finalproduction`.

Each model now has:

* A **version number**
* **Aliases** (like `@appserver`)
* **Timestamp** and **creator details**

This indicates that models are now properly versioned and ready for deployment in a production environment.

---

## **Conclusion**

In this notebook, I explored the concept of **Model Versioning** using **MLflow’s Model Registry**, which builds upon the experiment tracking concepts learned earlier.

I trained multiple models (Logistic Regression, Random Forest, and XGBoost with/without SMOTE) and logged their performance in MLflow.

After comparing results, the best model was **registered** in the MLflow **Model Registry**, creating a **versioned entry** that can be promoted, tracked, and deployed.

I also learned how to assign **aliases** (such as `@appserver`) for better management of staging and production models.

Overall, this exercise demonstrated how MLflow simplifies the **end-to-end workflow** from experiment tracking to **production-ready model management**.


## **Key Learnings**

* Understood how to use **MLflow’s Model Registry** to register, store, and manage models.
* Learned that each new registered model automatically gets a **version number** (v1, v2, etc.).
* Explored how to **compare multiple models** and select the best performing one through the MLflow UI.
* Practiced adding **aliases** to identify models in different environments (e.g., `@staging`, `@production`).
* Observed how MLflow ensures **traceability, reproducibility, and deployment readiness** in an MLOps workflow.
* Gained hands-on experience in moving from **experiment tracking (Day 67)** to **model versioning (Day 68)** — a key step toward scalable MLOps pipelines.

---
